In [1]:
import os
import numpy as np
import board
import pickle
import board as Board
import player
import game
from copy import deepcopy
import mcts 

from neural_network import*

Using TensorFlow backend.


In [ ]:
import sys

sys.path

In [2]:
player1 = player.Zero_Player('x', 'Bot_ZERO', nn_type="w", temperature=1)
player2 = player.Zero_Player('o', 'Bot_ZERO', nn_type="w", temperature=1)

In [5]:
brd = b.Board()
tree = mcts.MCTS()
PATH = {"path":[]}


In [ ]:
np.where(brd.board.ravel()!=0)[0].shape[0]

In [6]:
players = [player1, player2]
test = np.array([[ 1, -1,  1],
       [ 0,  1, -1],
       [ 1, 0, -1]])
turn = 5
n_map = {-1:"o", 0: " ", 1: "x"}

str_state = ''.join([n_map[i] for i in test.reshape(9,)])
str_state

brd.board = test

mcts.simulate(players, turn, brd, tree, PATH, root = brd, n = 1)

tree.update_nodes()

In [ ]:
root_node = get_node('xox xox o', tree.TREE)
root_node.N+=1

In [7]:
tree.TREE

{'xox xox o': <mcts.Node at 0x7f9d012aba58>,
 'xooox x x': <mcts.Node at 0x7f9d012ab9e8>,
 'o xox xox': <mcts.Node at 0x7f9d012ab9b0>,
 'x x xooox': <mcts.Node at 0x7f9d012aba20>}

In [ ]:
zero = [x for x in tree.TREE.values() if x.N==0]

for node in zero:
    print(node)
    print(node.board_arr)
    print(node.id)
    print(node.N, node.W, node.Q, node.D, node.L)

In [ ]:
root = [x for x in tree.TREE.values() if x.is_root()][0]

for x in [root]:
    print(x)
    print(x.board_arr)
    print(x.id)
    print(x.edges)
    print("W:",x.W, "L",x.L, "D", x.D)
    print("N:", x.N)
    print("Q",x.Q)

In [ ]:
for x in tree.TREE.values():
    print(x)
    print(x.board_arr)
    print(x.id)
    print(x.edges)
    print("W:",x.W, "L",x.L, "D", x.D)
    print("N:", x.N)
    print("Q",x.Q)


In [ ]:
tree.TREE

In [ ]:
player2.tree = tree.TREE
player2.turn(brd)